In [ ]:
from src.Model import Model
import torch
import torch.nn.functional as F
from tqdm import tqdm
from src.Tools import generate_text,get_eos_embedding,update_sequence_mask,get_one_hot
from src.Train import one_epoch_caption,one_epoch_QA,train_caption,train_QA
from src.Dataset import QADataset,CaptionDataset
from torch.utils.data import DataLoader
from src.Mean import ImagesDataset

version = 1
model = Model(model='BERT')

In [ ]:
QA_dataset = QADataset('Data/Annotations/QA/train.json',small=True)
Caption_dataset = CaptionDataset('Data/Annotations/Caption/train.json',small=True)
data_loader = DataLoader(Caption_dataset, batch_size= 2, shuffle = True)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = train_caption(4,model,data_loader,optimizer,version=version+1)